#nanoGPT 실행 및 활용
1. shakespeare 작품 dataset으로 GPT train 및 generation 
2. finetuning하여 gpt2-xl train 및 generation 
3. shakespeare 프로그램 변형하여 songwriter dataset으로 GPT train 및 generation 

프로그램 환경 : ColabPro+


#**Install**
transformers : load GPT-2 checkpoints를 위해 huggingface transformers  install

datasets :  OpenWebText를 다운로드하고 전처리(preprocess) 할때 필요

tiktoken : OpenAI's fast BPE code 사용

wandb : optional logging

In [1]:
!pip install transformers
!pip install datasets
!pip install tiktoken
!pip install wandb
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 21.0 MB/s eta 0:00:0

In [2]:
import os

### github에서 다운로드 한 nanoGPT 소스 파일 cloudServer에 load

In [3]:
# set the name of the folder to download images
img_folder_nm = 'nanoGPT'

# set the name of the zip file to be downloaded
img_zipfile_nm = 'nanoGPT.zip'

# dropbox link for images
dropbox_link = 'https://www.dropbox.com/sh/vaewl01pa8ewf26/AABKn6u0CDSn5JG3EHd6nvPXa?dl=0'

In [4]:
!wget -O {img_zipfile_nm} {dropbox_link}

--2023-04-24 01:14:52--  https://www.dropbox.com/sh/vaewl01pa8ewf26/AABKn6u0CDSn5JG3EHd6nvPXa?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.7.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.7.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /sh/raw/vaewl01pa8ewf26/AABKn6u0CDSn5JG3EHd6nvPXa [following]
--2023-04-24 01:14:53--  https://www.dropbox.com/sh/raw/vaewl01pa8ewf26/AABKn6u0CDSn5JG3EHd6nvPXa
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1e46be0ebdd5c19d1bc419a08d.dl.dropboxusercontent.com/zip_download_get/BeJONjmQw03UjIYShn4PXvmlZQbAwTBsuYM19LLld-BayBfdNIwkcQ-HjU0SKm5VRw3HyMx8xsupKqRb1piOcmdFBN9OICf3pzCVo_c8-e1dmA# [following]
--2023-04-24 01:14:54--  https://uc1e46be0ebdd5c19d1bc419a08d.dl.dropboxusercontent.com/zip_download_get/BeJONjmQw03UjIYShn4PXvmlZQbAwTBsuYM19LLld-BayBfdNIwkcQ-HjU0SKm5VRw3HyMx8xsupKqRb1piOcmdFBN9

In [5]:
ls

nanoGPT.zip  sample_data/


In [6]:
%%capture

!unzip {img_zipfile_nm} -d {img_folder_nm}

In [7]:
!ls {img_folder_nm}

assets	  configurator.py  model.py   scaling_laws.ipynb
bench.py  data		   README.md  train.py
config	  LICENSE	   sample.py  transformer_sizing.ipynb


#**quick start**

## prepare.py  : 
Shakespeare 작품을 가지고 데이터 준비
1. data load : Shakespeare 작품을 가지고 character-level GPT를 훈련
2. 다운로드 shakespeare dataset : 데이터 세트 문자 길이  1,115,394
3. 모든 고유 문자 갯수 : 65
4. train과 test data set 분리 : 
5. train과 test data tokenize :  train has 1,003,854 tokens / val has 111,540 tokens

  train_ids = encode(train_data)

  val_ids = encode(val_data)

5. train과 test data Token bin 파일 생성 :  to help us encode/decode:  train.bin, val.bin 파일
6. encode/decode를 위해 활용할 meta 파일 생성성

In [8]:
%run /content/nanoGPT/data/shakespeare_char/prepare.py 

length of dataset in characters: 1,115,394
all the unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65
train has 1,003,854 tokens
val has 111,540 tokens


 위 prepare.py 실행 후 train.bin, val.bin 파일이 생성됨을 확인 할 수 있다.

In [9]:
!ls /content/nanoGPT/data/shakespeare_char/

input.txt  meta.pkl  prepare.py  readme.md  train.bin  val.bin


In [10]:
os.chdir('nanoGPT/')

## train.py : 
config/train_shakespeare_char.py config 파일을 이용해 GPT train

1. train 할 gpt2의 config values setting 

  [data] dataset = 'shakespeare_char' / 
batch_size = 64  / 
block_size = 256

  [GPT model] 
n_layer = 6 /
n_head = 6 /
n_embd = 384 /
dropout = 0.2  

  [learning rate decay settings] learning_rate = 1e-3  # max learning rate / 
max_iters = 5000  # total number of training iterations   /
lr_decay_iters = 5000  /
min_lr = 1e-4    # minimum learning rate

2. GPT train을 위해 지정된 configurator.py을 불러옴 (train_shakespeare_char.py)

  config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]

  exec(open('configurator.py').read()) # overrides from command line or config file

  config = {k: globals()[k] for k in config_keys} # will be useful for logging

3. various inits, derived attributes, I/O setup : 
total number of tokens per iteration (batch_size * block_size * radient_accumulation_steps ) : 655360

4. model init

    gptconf = GPTConfig(**model_args) : vocab_size = 65 

    model = GPT(gptconf)

5. compile the model

  training loop :  
  5.1 determine and set the learning rate for this iteration 
  
  5.2 evaluate the loss on train/val sets and write checkpoints

6.iteration 5000 수행 후 최적의  train loss / val loss : step 5000: train loss 0.0715, val loss 3.7191

In [11]:
%run train.py config/train_shakespeare_char.py

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
batch_size = 64
block_size = 256 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of tokens per iter is small

warmup

[2023-04-24 01:16:20,546] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-04-24 01:16:21,171] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-04-24 01:16:22,148] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-04-24 01:16:22,494] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-04-24 01:16:22,961] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-04-24 01:16:23,325] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-04-24 01:16:23,794] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-04-24 01:16:24,177] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-04-24 01:16:24,651] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-04-24 01:16:2

iter 0: loss 4.2616, time 27805.33ms, mfu -100.00%
iter 10: loss 3.2220, time 940.45ms, mfu 15.85%
iter 20: loss 2.7700, time 927.58ms, mfu 15.87%
iter 30: loss 2.6168, time 922.60ms, mfu 15.90%
iter 40: loss 2.5380, time 942.05ms, mfu 15.89%
iter 50: loss 2.5132, time 951.66ms, mfu 15.87%
iter 60: loss 2.4730, time 939.12ms, mfu 15.87%
iter 70: loss 2.4802, time 924.66ms, mfu 15.89%
iter 80: loss 2.4408, time 961.47ms, mfu 15.85%
iter 90: loss 2.4262, time 934.52ms, mfu 15.86%
iter 100: loss 2.4047, time 929.98ms, mfu 15.88%
iter 110: loss 2.3830, time 958.49ms, mfu 15.85%
iter 120: loss 2.3723, time 929.44ms, mfu 15.87%
iter 130: loss 2.3096, time 934.14ms, mfu 15.88%
iter 140: loss 2.2582, time 947.57ms, mfu 15.86%
iter 150: loss 2.1338, time 935.40ms, mfu 15.87%
iter 160: loss 2.0861, time 931.74ms, mfu 15.88%
iter 170: loss 2.0113, time 933.80ms, mfu 15.89%
iter 180: loss 1.9555, time 975.49ms, mfu 15.83%
iter 190: loss 1.8550, time 923.75ms, mfu 15.86%
iter 200: loss 1.8575, time

## sample.py : 
train 후 sample.py로 GPT 테스트 

1. 하이퍼 파라메터

   num_samples = 10 # number of samples to draw / 
max_new_tokens = 500 # number of tokens generated in each sample  /
temperature = 0.8 # 는 생성된 단어의 다양성을 제어하는 역할 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions / 
top_k = 200 # 후보 단어 집합에서 선택할 단어의 개수를 제한하는 역할 retain only the top_k most likely tokens, clamp others to have 0 probability / 
seed = 1337

2. model : GPT 설정 후 모델의 파라미터들을 불러옴.
    model = GPT(gptconf)

    state_dict = checkpoint['model']
  
    model.load_state_dict(state_dict)

3. model.eval() : 모델의 파라미터들이 고정된 상태로 설정되고, dropout과 batch normalization과 같은 연산들도 eval 상태로 변경

 메타 파일은 data/shakespeare_char/meta.pkl을 사용

4. 자연어 생성 :  
  num_samples 10개 만큼 , max_new_tokens = 500, temperature = 0.8, top_k = 200
   
   for k in range(num_samples): y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)



In [12]:
%run sample.py --out_dir=out-shakespeare-char

Overriding: out_dir = out-shakespeare-char
number of parameters: 10.65M
Loading meta from data/shakespeare_char/meta.pkl...


KING RICHARD II:
Say they stay a service, and all the streaginates:
Whither 'twas he that did him enjoy?

DUCHESS OF YORK:
What with the way come matter?

KING RICHARD III:
Old George overthanks, methinks, that we learn you with us.

DUCHESS OF YORK:
My woyfully sister for this death:
I will make keep Warwick, their king arm down.

YORK:
I can poor great to hear him, they find;
And he must not know him from Prince, Somerset,
Let's too dischard a curse of the discontrary
And his hand, and for hi
---------------

Menenius, sir, you shall have hand her;
And the noble two men reason; and being with weeping,
The score something casely to England,
My worst shall be beloved to die.

PRINCE:
And I cannot see the king in my lands:
I come, sir, sir, then it die now, for the child.

HENRY BOLINGBROKE:
Well, then long a despised by my life;
For he shall desire be the napkin

#**finetuning**
미세조정은 train과 다르지 않고 pre-trained 된 모델을 초기화하고 더 작은 학습 속도로 train 한다.

## train.py
1. confing file : finetune_shakespeare.py
  
  1.1 eval_interval = 5 / 
eval_iters = 40 /
dataset = 'shakespeare'
init_from = 'gpt2-xl' # this is the largest GPT-2 model

  1.2 total number of tokens per iteration (batch_size * block_size * radient_accumulation_steps ) : 1 batch_size * 32 grad_accum * 1024 tokens = 32,768 tokens/iter --> 실제 아래 train.py 수행 시는 config 파일 주석과 달리 262,144 개가 수행 됨

  shakespeare 301,966 tokens 있어서 1 epoch ~= 9.2 iteratjons 수행행

  batch_size = 1 / 
gradient_accumulation_steps = 32 / 
max_iters = 20

  1.3 finetune at constant LR : 
learning_rate = 3e-5  / 
decay_lr = False

2. train.py : config/finetune_shakespeare.py config 파일을 이용해 gpt2-xl train

 2.1 위의 'quick start'에서 생성된 train.bin과 val.bin를 복사 해서 다운로드 해 둔다.

 2.2 위의 'quick start'의 train과 같은 프로그램인 train.py를 finetune_shakespeare.py로 finetuning하여 수행

 total number of tokens per iteration: 262144

 OpenAI GPT-2 weights: gpt2-xl

 forcing vocab_size=50257, block_size=1024, bias=True

 number of parameters: 1555.97M

 using fused AdamW: True 
 - AdamW는 Adam(Adaptive Moment Estimation) 옵티마이저의 변형으로, 가중치 감쇠(weight decay)를 수행하는 방법이며 Adam의 가중치 감쇠와 다르게, 가중치 감쇠를 weight decay만으로 수행하지 않고, 가중치 감쇠와 optimizer의 파라미터 감쇠를 결합하여 수행합니다. 이를 통해 모델 파라미터의 편향(bias)에 대한 가중치 감쇠를 수행하는 동시에, 가중치 감쇠를 수행하는 것과 optimizer의 파라미터 감쇠를 따로 적용하는 것보다 더욱 좋은 성능을 보임

2.3 iteration 20번 실행 후 최적의  train loss / val loss : step 20 : train loss 1.9367, val loss 1.9109


'quick start'에서 생성된 train.bin과 val.bin를 복사 해서 다운로드 해 둔다.

In [14]:
import shutil

shutil.copy('/content/nanoGPT/data/shakespeare_char/train.bin', '/content/nanoGPT/data/shakespeare/')
shutil.copy('/content/nanoGPT/data/shakespeare_char/val.bin', '/content/nanoGPT/data/shakespeare/')

'/content/nanoGPT/data/shakespeare/val.bin'

In [15]:
!ls /content/nanoGPT/data/shakespeare/

prepare.py  readme.md  train.bin  val.bin


In [16]:
%run train.py config/finetune_shakespeare.py

Overriding config with config/finetune_shakespeare.py:
import time

out_dir = 'out-shakespeare'
eval_interval = 5
eval_iters = 40
wandb_log = False # feel free to turn on
wandb_project = 'shakespeare'
wandb_run_name = 'ft-' + str(time.time())

dataset = 'shakespeare'
init_from = 'gpt2-xl' # this is the largest GPT-2 model

# only save checkpoints if the validation loss improves
always_save_checkpoint = False

# the number of examples per iter:
# 1 batch_size * 32 grad_accum * 1024 tokens = 32,768 tokens/iter
# shakespeare has 301,966 tokens, so 1 epoch ~= 9.2 iters
batch_size = 1
gradient_accumulation_steps = 32
max_iters = 20

# finetune at constant LR
learning_rate = 3e-5
decay_lr = False

total number of tokens per iteration: 262144
Initializing from OpenAI GPT-2 weights: gpt2-xl
loading weights from pretrained gpt: gpt2-xl
forcing vocab_size=50257, block_size=1024, bias=True
overriding dropout rate to 0.0
number of parameters: 1555.97M


using fused AdamW: True
compiling the model... (takes a ~minute)
step 0: train loss 3.8766, val loss 3.7893


[2023-04-24 02:52:12,330] torch._dynamo.convert_frame: [WARNING] torch._dynamo hit config.cache_size_limit (64)
   function: 'forward' (/content/nanoGPT/model.py:110)
   reasons:  ___check_obj_id(self, 140596194586000)
to diagnose recompilation issues, see https://pytorch.org/docs/master/dynamo/troubleshooting.html.
[2023-04-24 02:52:41,014] torch._dynamo.convert_frame: [WARNING] torch._dynamo hit config.cache_size_limit (64)
   function: 'forward' (/content/nanoGPT/model.py:34)
   reasons:  ___check_obj_id(self, 140596194996960)
to diagnose recompilation issues, see https://pytorch.org/docs/master/dynamo/troubleshooting.html.


iter 0: loss 3.8493, time 130696.06ms, mfu -100.00%
iter 1: loss 3.2312, time 37948.27ms, mfu -100.00%
iter 2: loss 3.1915, time 37796.50ms, mfu -100.00%
iter 3: loss 2.8811, time 37874.83ms, mfu -100.00%
iter 4: loss 2.5588, time 38048.24ms, mfu -100.00%


[2023-04-24 02:56:23,267] torch._dynamo.convert_frame: [WARNING] torch._dynamo hit config.cache_size_limit (64)
   function: 'forward' (/content/nanoGPT/model.py:60)
   reasons:  ___check_obj_id(self, 140596194586288)
to diagnose recompilation issues, see https://pytorch.org/docs/master/dynamo/troubleshooting.html.
[2023-04-24 02:56:24,523] torch._dynamo.convert_frame: [WARNING] torch._dynamo hit config.cache_size_limit (64)
   function: 'forward' (/content/nanoGPT/model.py:94)
   reasons:  ___check_obj_id(self, 140594753651376)
to diagnose recompilation issues, see https://pytorch.org/docs/master/dynamo/troubleshooting.html.


step 5: train loss 2.5800, val loss 2.5622
saving checkpoint to out-shakespeare
iter 5: loss 2.5740, time 102855.24ms, mfu 8.40%
iter 6: loss 2.5469, time 39792.13ms, mfu 9.73%
iter 7: loss 2.3779, time 39777.31ms, mfu 10.93%
iter 8: loss 2.2152, time 39777.26ms, mfu 12.01%
iter 9: loss 2.3448, time 39783.47ms, mfu 12.98%
step 10: train loss 2.2956, val loss 2.2889
saving checkpoint to out-shakespeare
iter 10: loss 2.3615, time 101504.21ms, mfu 12.53%
iter 11: loss 2.2208, time 39789.24ms, mfu 13.45%
iter 12: loss 2.2204, time 39771.99ms, mfu 14.27%
iter 13: loss 2.2598, time 39769.26ms, mfu 15.02%
iter 14: loss 2.1561, time 39782.97ms, mfu 15.69%
step 15: train loss 2.1212, val loss 2.0522
saving checkpoint to out-shakespeare
iter 15: loss 2.1180, time 101603.48ms, mfu 14.97%
iter 16: loss 2.1617, time 39789.15ms, mfu 15.64%
iter 17: loss 2.0797, time 39782.52ms, mfu 16.25%
iter 18: loss 2.1993, time 39777.65ms, mfu 16.80%
iter 19: loss 2.0556, time 39798.95ms, mfu 17.29%
step 20: tra

##finetuning Sample : 
finetuning 모델을 가지고 "What is the answer to life, the universe, and everything?" 질의에 대한 생성

- 모델 설정 : init_from = gpt2-xl (the largest GPT-2 model) 
- 질의문장 : start = What is the answer to life, the universe, and everything? 
- 생성한 sample 수 : num_samples = 5
- 각 샘플의 최대 생성 단어 : max_new_tokens = 100

In [17]:
%run sample.py \
    --init_from=gpt2-xl \
    --start="What is the answer to life, the universe, and everything?" \
    --num_samples=5 --max_new_tokens=100

Overriding: init_from = gpt2-xl
Overriding: start = What is the answer to life, the universe, and everything?
Overriding: num_samples = 5
Overriding: max_new_tokens = 100
loading weights from pretrained gpt: gpt2-xl
forcing vocab_size=50257, block_size=1024, bias=True
overriding dropout rate to 0.0
number of parameters: 1555.97M
No meta.pkl found, assuming GPT-2 encodings...
What is the answer to life, the universe, and everything?

One possibility is that they have a universe-wide perspective that allows for no contradictions. And, if so, then why is there contradiction?

Another possibility is that the universe is one big joke. They are not the only joke in the universe. But it is not an empty universe.

Hence the problems with the universe's Big Bang theory, which tells us that the universe started out in a singularity with a zero initial mass. The Big Bang is essentially consistent with
---------------
What is the answer to life, the universe, and everything? Probably not. Quantum 

#**nanoGPT 활용해서 songWriter 작성해 보기**

# dataset : 
 - **Kaggle's Spotify Million Song Dataset** 사용 : spotify_millsongdata.csv
 - artist name, song name, link : song and lyrics, text

## prepare.py : 

/data/shakespeare/prepare.py 변형

songwriter를 위한 dataset file load : spotify_millsongdata.csv
 - 해당 엑셀의 'text' 컬럼으로 data 추출하여 train, val data 사용

In [21]:
%run /content/nanoGPT/data/songwriter/prepare.py 

train has 22,308,928 tokens
val has 2,456,916 tokens


위 prepare.py 실행 후 train.bin, val.bin 파일이 생성됨을 확인 할 수 있다.

In [22]:
!ls /content/nanoGPT/data/songwriter/

prepare.py  spotify_millsongdata.csv  train.bin  val.bin


## train.py : 
config/train_shakespeare_char.py config 파일을 이용해 GPT train

iteration 5000 번 수행 후 최적의 train loss/val loss : step 5000 train loss 1.6726, val loss 2.3793

In [24]:
%run train.py config/train_songwriter.py

Overriding config with config/train_songwriter.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-songwriter'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'songwriter'
wandb_run_name = 'mini-gpt'

dataset = 'songwriter'
batch_size = 64
block_size = 256 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of tokens per iter is small

warmup_iters = 100 # not super

#sample.py :
 - LOVE를 질의하여 노래가사 생성 
 - number of parameters: 29.94M
 - GPT-2 encodings

In [25]:
%run sample.py --out_dir=out-songwriter --start="LOVE" 

Overriding: out_dir = out-songwriter
Overriding: start = LOVE
number of parameters: 29.94M
No meta.pkl found, assuming GPT-2 encodings...
LOVE ME !  
  
  
And it's beautiful  
Forget about your face  
I'll fill the air  
I can't believe it  
That I'm ok  
I will be ok  
And I will be ok  
And I will be ok  
  
I'm ok  
I'm ok  
  
So come on  
I'm ok  
I'm ok  
I'm ok  
I'm ok  
I'm ok  
  
I'm ok  
I'm ok  
I'm ok  
I'm ok  
I'm ok  
  
I'm ok  
I'm ok  
I'm ok  
I'm ok  
I'm ok


I ain't just for you  
They don't come without me  
  
Nobody lives in the world for me  
Nobody lives in the world for me  
Nobody lives in the world for me  
Nobody lives in the world for me  
  
It's not like you  
It's not like you  
It's not like you  
It's not like you  
It's not like you  
  
It's not like you  
It's not like you  
It's not like you  
It's not like you  
It's not like you  
It's not like you  
It's not like you  
It's not like you  
It's not like you  
It's not like you  
It's not li